# Data Audit

This script takes an audit of the EPC data for Wales. It looks at how many missing values each feature has, how many unique values the categorical features have then applies preprocessing steps to remove outliers and drop features with too many missing values.

In [1]:
import numpy as np
import pandas as pd
import os
import json
import matplotlib.pyplot as plt

In [3]:
# set variables from config file
config_path = os.path.abspath('..')

with open(config_path + '/config.json', 'r') as f:
    config = json.load(f)

processing_path = config['DEFAULT']['processing_path']
epc_wales_fname = config['DEFAULT']['epc_wales_fname']
epc_train_fname = config['DEFAULT']['epc_train_fname']
epc_test_fname = config['DEFAULT']['epc_test_fname']

In [4]:
#Specify ther import dictionary for the dates
dtype_dict = {'INSPECTION_DATE':'str'}

epc_train = pd.read_csv(os.path.join(processing_path,epc_train_fname), header = 0, delimiter = ',', 
                       dtype = dtype_dict, parse_dates = ['INSPECTION_DATE'])

epc_test = pd.read_csv(os.path.join(processing_path,epc_test_fname),header = 0, delimiter = ',',
                        dtype = dtype_dict, parse_dates = ['INSPECTION_DATE'])

In [5]:
epc_train['inspection_year'] = epc_train['INSPECTION_DATE'].dt.year

In [6]:
epc_train['inspection_year'].value_counts(normalize=True)

2022    0.995737
2021    0.004263
Name: inspection_year, dtype: float64

In [7]:
print(len(epc_train))
print(len(epc_test))

3988
998


### Investigate categoric and numeric fields

In [8]:
# Get numeric and categorical variable names 
var_list_num = epc_train.select_dtypes(include= 'number').columns.tolist()

var_list_cat = epc_train.select_dtypes(include= ['object','category']).columns.tolist()
var_list_cat.remove('LMK_KEY')
var_list_cat.remove('POSTCODE')

In [9]:
# Percentage of missing values in numeric fields
epc_train[var_list_num].isnull().sum().sort_values(ascending = False) / epc_train.shape[0]

FLAT_STOREY_COUNT                1.000000
MAIN_HEATING_CONTROLS            1.000000
LOW_ENERGY_FIXED_LIGHT_COUNT     1.000000
UNHEATED_CORRIDOR_LENGTH         1.000000
HEAT_LOSS_CORRIDOR               1.000000
SHEATING_ENV_EFF                 1.000000
SHEATING_ENERGY_EFF              1.000000
FLOOR_LEVEL                      1.000000
PHOTO_SUPPLY                     0.136409
NUMBER_HEATED_ROOMS              0.125376
NUMBER_HABITABLE_ROOMS           0.125376
EXTENSION_COUNT                  0.125376
UPRN                             0.081996
CO2_EMISSIONS_POTENTIAL          0.000000
CO2_EMISS_CURR_PER_FLOOR_AREA    0.000000
CO2_EMISSIONS_CURRENT            0.000000
ENERGY_CONSUMPTION_CURRENT       0.000000
ENERGY_CONSUMPTION_POTENTIAL     0.000000
LIGHTING_COST_POTENTIAL          0.000000
ENVIRONMENT_IMPACT_POTENTIAL     0.000000
ENVIRONMENT_IMPACT_CURRENT       0.000000
POTENTIAL_ENERGY_EFFICIENCY      0.000000
CURRENT_ENERGY_EFFICIENCY        0.000000
LIGHTING_COST_CURRENT            0

Drop COUNTY, FLAT_STOREY_COUNT AND UNHEATED_CORRIDOR_LENGTH due to high percentage of 
missing values. Keep FLOOR_HEIGHT but be careful

In [10]:
audit_num = epc_train[var_list_num].describe(percentiles = [0.25,0.5,0.75,0.9,0.95,0.99])

We have some negative values and some massively large

Clip all below 0

Clip all at the 95th percentile with the exception of the following

In [11]:
# Percentage of missing values in categoric fields
epc_train[var_list_cat].isnull().sum().sort_values(ascending = False) / epc_train.shape[0]

ADDRESS3                    0.946339
FLOOR_ENERGY_EFF            0.874624
FLOOR_ENV_EFF               0.874624
FLAT_TOP_STOREY             0.874624
COUNTY                      0.625878
ADDRESS2                    0.619358
GLAZED_TYPE                 0.167001
MECHANICAL_VENTILATION      0.125376
SOLAR_WATER_HEATING_FLAG    0.125376
GLAZED_AREA                 0.125376
MAINS_GAS_FLAG              0.125376
ENERGY_TARIFF               0.116098
UPRN_SOURCE                 0.081996
LOCAL_AUTHORITY_LABEL       0.010030
POSTTOWN                    0.001755
ROOF_ENV_EFF                0.000251
WINDOWS_ENV_EFF             0.000251
WINDOWS_ENERGY_EFF          0.000251
WINDOWS_DESCRIPTION         0.000251
ROOF_ENERGY_EFF             0.000251
HOTWATER_DESCRIPTION        0.000000
HOT_WATER_ENERGY_EFF        0.000000
HOT_WATER_ENV_EFF           0.000000
region                      0.000000
FLOOR_DESCRIPTION           0.000000
LODGEMENT_DATE              0.000000
CONSTITUENCY                0.000000
L

Drop SHEATING_ENERGY_EFF, SHEATING_ENV_EFF, FLOOR_ENV_EFF, FLAT_TOP_STOREY, FLOOR_ENERGY_EFF
and SOLAR_WATER_HEATING_FLAG due to percentage of missing values

In [12]:
audit_cat = epc_train[var_list_cat].describe()
epc_train[var_list_cat].nunique().sort_values(ascending = False)

ADDRESS                     3985
ADDRESS1                    3971
LODGEMENT_DATETIME          3866
ADDRESS2                    1148
POSTTOWN                     892
CONSTITUENCY_LABEL           566
CONSTITUENCY                 566
LOCAL_AUTHORITY              325
LOCAL_AUTHORITY_LABEL        323
ADDRESS3                     193
LIGHTING_DESCRIPTION          93
MAINHEAT_DESCRIPTION          52
ROOF_DESCRIPTION              49
WALLS_DESCRIPTION             49
FLOOR_DESCRIPTION             36
COUNTY                        24
MAINHEATCONT_DESCRIPTION      23
CONSTRUCTION_AGE_BAND         21
MAIN_FUEL                     18
HOTWATER_DESCRIPTION          17
WINDOWS_DESCRIPTION           14
SECONDHEAT_DESCRIPTION        12
TRANSACTION_TYPE              11
ENERGY_TARIFF                  8
CURRENT_ENERGY_RATING          7
BUILT_FORM                     6
GLAZED_TYPE                    6
HOT_WATER_ENERGY_EFF           5
LIGHTING_ENERGY_EFF            5
GLAZED_AREA                    5
POTENTIAL_

We need to reduce the levels

### Drop unwanted fields

In [13]:
epc_train.drop(['COUNTY','FLAT_STOREY_COUNT','UNHEATED_CORRIDOR_LENGTH','LODGEMENT_DATE',
                'SHEATING_ENERGY_EFF','SHEATING_ENV_EFF','FLOOR_ENV_EFF',
                'FLAT_TOP_STOREY','FLOOR_ENERGY_EFF','SOLAR_WATER_HEATING_FLAG'],
                axis = 1,
                inplace = True)

epc_test.drop(['COUNTY','FLAT_STOREY_COUNT','UNHEATED_CORRIDOR_LENGTH','LODGEMENT_DATE',
               'SHEATING_ENERGY_EFF','SHEATING_ENV_EFF','FLOOR_ENV_EFF',
               'FLAT_TOP_STOREY','FLOOR_ENERGY_EFF','SOLAR_WATER_HEATING_FLAG'],
               axis = 1,
               inplace = True)

### Clip training and test datasets

In [14]:
def call_clip(df):
    
    ''' 
    Clips the variables of the EPC data between 0 and a specified upper amount
    Parameters
      df: a dataframe containing EPC data specified below
    Returns the dataframe with clipped values
    '''
    
    df['CURRENT_ENERGY_EFFICIENCY'].clip(lower = 0.0, upper = 100, inplace = True)
    df['ENERGY_CONSUMPTION_CURRENT'].clip(lower = 0.0, upper = 600, inplace = True)
    df['CO2_EMISSIONS_CURRENT'].clip(lower = 0.0, upper = 20, inplace = True)
    df['CO2_EMISS_CURR_PER_FLOOR_AREA'].clip(lower = 0.0, upper = 100, inplace = True)
    df['LIGHTING_COST_CURRENT'].clip(lower = 0.0, upper = 150, inplace = True)
    df['HEATING_COST_CURRENT'].clip(lower = 0.0, upper = 1750, inplace = True)
    df['HOT_WATER_COST_CURRENT'].clip(lower = 0.0, upper = 350, inplace = True)
    df['TOTAL_FLOOR_AREA'].clip(lower = 0.0, upper = 175, inplace = True)
    df['MULTI_GLAZE_PROPORTION'].clip(lower = 0.0, upper = 100, inplace = True)
    df['EXTENSION_COUNT'].clip(lower = 0.0, upper = 4, inplace = True)
    df['NUMBER_HABITABLE_ROOMS'].clip(lower = 0.0, upper = 10, inplace = True)
    df['NUMBER_HEATED_ROOMS'].clip(lower = 0.0, upper = 9, inplace = True)
    df['LOW_ENERGY_LIGHTING'].clip(lower = 0.0, upper = 100, inplace = True)
    df['NUMBER_OPEN_FIREPLACES'].clip(lower = 0.0, upper = 4, inplace = True)
    df['FLOOR_HEIGHT'].clip(lower = 0.0, upper = 3, inplace = True)
    return(df)

epc_train = call_clip(epc_train)
epc_test = call_clip(epc_test)

### Export datasets

In [15]:
epc_train.to_csv(os.path.join(processing_path,epc_train_fname),index = False)
epc_test.to_csv(os.path.join(processing_path,epc_test_fname),index = False)